In [ ]:
repo_dir = "Repos"   # Set this to be where your github repos are located.
%load_ext autoreload
%autoreload 2

# Update the load path so python can find modules for the model
import sys
from pathlib import Path
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))

In [ ]:
# Prerequisites

import json
import os
from eye_ai.eye_ai import EyeAI
import pandas as pd
from pathlib import Path, PurePath
import logging
# import torch

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [16]:
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
catalog_id = "eye-ai" #@param
host = 'dev.eye-ai.org'
# host = 'www.eye-ai.org'


gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

2024-06-10 17:22:57,665 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-06-10 17:22:57,665 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


You are already logged in.


In [17]:
# Variables to configure the rest of the notebook.

cache_dir = '/data'        # Directory in which to cache materialized BDBags for datasets
working_dir = '/data'    # Directory in which to place output files for later upload.

configuration_rid="2-BH84" # rid dev: 2-BH84 prod: 2-C60W


In [18]:
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

2024-06-10 17:23:01,726 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-06-10 17:23:01,727 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2024-06-10 17:23:01,898 - INFO - Loading dirty model.  Consider commiting and tagging: 1.1.0.post18+git.d89701e2.dirty


In [ ]:
# @title Initiate an Execution
configuration_records = EA.execution_init(configuration_rid=configuration_rid)
configuration_records.model_dump()

2024-06-10 17:23:03,779 - INFO - File [/data/lizhiwei/EyeAI_working/Execution_Metadata/Execution_Config-multimodal_create_cond_label.json] transfer successful. 0.60 KB transferred. Elapsed time: 0:00:00.000050.
2024-06-10 17:23:03,780 - INFO - Verifying MD5 checksum for downloaded file [/data/lizhiwei/EyeAI_working/Execution_Metadata/Execution_Config-multimodal_create_cond_label.json]
2024-06-10 17:23:03,812 - INFO - Configuration validation successful!
2024-06-10 17:23:04,233 - INFO - Attempting to resolve http://identifiers.org/minid:Shus6unyYcrF into a valid set of URLs.
2024-06-10 17:23:05,072 - INFO - The identifier minid:Shus6unyYcrF resolved into the following locations: [https://eye-ai-shared.s3.amazonaws.com//6656e08709b2f7c0bbed8284fdfb550d/2024-06-11_00.19.45/Dataset_2-6162.zip]
2024-06-10 17:23:05,075 - INFO - Attempting GET from URL: https://eye-ai-shared.s3.amazonaws.com//6656e08709b2f7c0bbed8284fdfb550d/2024-06-11_00.19.45/Dataset_2-6162.zip
2024-06-10 17:23:05,329 - INF

In [ ]:
# Compute condition labels
icd10 = pd.read_csv(configuration_records.bag_paths[0]/'data/Clinic_ICD10.csv').drop(columns=['RCT', 'RMT', 'RCB', 'RMB'])
icd10_asso = pd.read_csv(configuration_records.bag_paths[0]/'data/Clinic_ICD_Asso.csv').drop(columns=['RCT', 'RMT', 'RCB', 'RMB'])

condition_labels = EA.compute_condition_label(icd10_asso, icd10)

In [ ]:
# Insert new condition labels
EA.insert_condition_label(condition_labels)


In [ ]:
uploaded_assets = EA.execution_upload(configuration_records.execution_rid, False)